 # Исследование надёжности заёмщиков — анализ банковских данных
 
 На основе статистики о платёжеспособности клиентов исследовать влияет ли семейное положение и количество детей клиента на факт возврата кредита в срок


## Описание проекта

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов. Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

# Исследование надежности заемщиков

1. [Обзор данных](#overwiev_id)
2. [Предобработка данных](#preprocessing_id)
    
    * [Обработка пропущенных значений](#preprocessing_lost_values_id)
    * [Проверка данных на аномалии и исправления](#chesk_anomaly_id)
        * [Удаление дубликатов](#remove_dublicates_id)
        * [Категоризация целей кредита](#target_cat_id)
        * [Категоризация дохода](#income_cat_id)

        * [Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма.
](#additional_dataframe_id)
3. [Ответы на вопросы](#answer_id)
4. [Общий вывод](#common_answer_id)


In [1]:
import pandas as pd

<a id="overwiev_id"></a>
## Шаг 1. Обзор данных 

In [2]:
data = pd.read_csv('/datasets/data.csv')

**Описание данных:**

- `children` — количество детей в семье
- `days_employed` — общий трудовой стаж в днях
- `dob_years` — возраст клиента в годах
- `education` — уровень образования клиента
- `education_id` — идентификатор уровня образования
- `family_status` — семейное положение
- `family_status_id` — идентификатор семейного положения
- `gender` — пол клиента
- `income_type` — тип занятости
- `debt` — имел ли задолженность по возврату кредитов
- `total_income` — ежемесячный доход
- `purpose` — цель получения кредита

Посмотрим на датасет для начала:

In [3]:
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


In [4]:
data.tail()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
21520,1,-4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем
21521,0,343937.404131,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,сделка с автомобилем
21522,1,-2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость
21523,3,-3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля
21524,2,-1984.507589,40,среднее,1,женат / замужем,0,F,сотрудник,0,82047.418899,на покупку автомобиля


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


При первичном анализе можно увидеть что столбцы days_employed и total_income имеют пропущенные значения, позже с этим надо разобраться

<a id="preprocessing_id"></a>
## Шаг 2. Предобработка данных  

<a id="preprocessing_lost_values_id"></a>
### Обработка пропущенных значений  

В столбцах `days_employed` и `total_income` пропущены значения. Разберемся с этим!
#### Обработка пропущенных значений в столбце `days_employed`

Рабочий стаж в дняx:

In [6]:
data['days_employed'].describe()

count     19351.000000
mean      63046.497661
std      140827.311974
min      -18388.949901
25%       -2747.423625
50%       -1203.369529
75%        -291.095954
max      401755.400475
Name: days_employed, dtype: float64

In [7]:
data['days_employed'] = abs(data['days_employed'])

In [8]:
data[data['days_employed'] > 100*365].groupby(by='income_type')["total_income"].count()

income_type
безработный       2
пенсионер      3443
Name: total_income, dtype: int64

In [9]:
print(f"Всего пенсионеров: {data[data['income_type'] == 'безработный']['days_employed'].count()}")
print(f"Всего пенсионеров: {data[data['income_type'] == 'пенсионер']['days_employed'].count()}")

Всего пенсионеров: 2
Всего пенсионеров: 3443


Данные по рабочему стажу не очень адекватны. 

Отрицательные значения легко объяснить объясняются ошибкой ввода данных и так же легко исправляются

У части записей рабочий стаж превышает 100 лет. Проведя некоторые изыскания, можно увидеть, что все *плохие* записи относятся к категориям столбца `income_type` `безработный` и  `пенсионер`. 

Еще один странный момент - испорчены **ВСЕ** данные по пенсионерам и безработным.

Если продолжить изыскания, можно увидеть, что все данные по этим категориям испорчены аддитивной ошибкой в диапазоне 900-1000 лет. 

*Можно предположить, что данные собирались после 2000 года и произошла путанница связанная с переходом от формата 19ХХ к 20ХХ. Но с другой стороны тогда должны были бы быть испорчены данные у людей схожего возраста, a-la сотрудник, но в их категории все хорошо.*

Идеальным вариантом было бы связаться с источником данных и узнать как собирались данные, как вглядит анкета для заполнения, например, etc.

В отсутствие идеального варианта, я заполню данные `income_type` с типом `безработный` и  `пенсионер` их средним значением, остальные же - средним значением по всему датасету, исключая `безработный` и  `пенсионер`

In [10]:
rule = (data['income_type'] != 'безработный')&(data['income_type'] != 'пенсионер')
stats_normal = data[rule]['days_employed'].describe()
stats_corrupt = data[~rule]['days_employed'].describe()

print('\n\nбезработный + пенсионер\n\n', stats_corrupt,  '\n\nвсе остальные\n\n', stats_normal, )




безработный + пенсионер

 count      3445.000000
mean     365004.309916
std       21075.016396
min      328728.720605
25%      346639.413916
50%      365213.306266
75%      383246.444219
max      401755.400475
Name: days_employed, dtype: float64 

все остальные

 count    15906.000000
mean      2353.015932
std       2304.243851
min         24.141633
25%        756.371964
50%       1630.019381
75%       3157.480084
max      18388.949901
Name: days_employed, dtype: float64


In [11]:
data.loc[rule, 'days_employed'] = data.loc[rule, 'days_employed'].fillna(stats_normal['mean'])
data.loc[~rule, 'days_employed'] = data.loc[~rule, 'days_employed'].fillna(stats_corrupt['mean'])

#### Обработка пропущенных значений в столбце `total_income`

Данные в столбце выглядит адекватно, но у нас наблюдаются нулевые значения дохода.

Строго говоря, в данном случае данные выглядят недостаточными. 
Сами по себе нулевые значения дохода не выглядят чем-то плохим - возможно отсутствие ежемесячного дохода, но наличие счета, акций, etc.

Поэтому логично заполнять пропуски нулем а не средним значением.

In [12]:
data['total_income'] = data['total_income'].fillna(0)
data['total_income'] = data['total_income'].astype(int)

In [13]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  int64  
 11  purpose           21525 non-null  object 
dtypes: float64(1), int64(6), object(5)
memory usage: 2.0+ MB


<a id="chesk_anomaly_id"></a>
### Проверка данных на аномалии и исправления

#### Количество детей

Количество детей 20, без наличия промежуточных значений от 5 до 20 выглядит недостоверным, количество детей -1, скорее всего просто является ошибкой ввода. Поправим:

In [14]:
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5])

In [15]:
data.groupby('children')['children'].count()

children
-1        47
 0     14149
 1      4818
 2      2055
 3       330
 4        41
 5         9
 20       76
Name: children, dtype: int64

In [16]:
data['children'].replace(-1, 1, inplace=True)
data['children'].replace(20, 2, inplace=True)

In [17]:
data['children'].unique()

array([1, 0, 3, 2, 4, 5])

#### Образование

В столбце образование - путанница с регистром, причина - ошибки ввода. Приведем все к нижнему регистру

In [18]:
data['education'].unique()

array(['высшее', 'среднее', 'Среднее', 'СРЕДНЕЕ', 'ВЫСШЕЕ',
       'неоконченное высшее', 'начальное', 'Высшее',
       'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Неоконченное высшее', 'НАЧАЛЬНОЕ',
       'Начальное', 'Ученая степень', 'УЧЕНАЯ СТЕПЕНЬ', 'ученая степень'],
      dtype=object)

In [19]:
data['education'] = data['education'].str.lower()

In [20]:
data['education'].unique()

array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

#### Семейный статус
Приведем все к нижнему регистру

In [21]:
data['family_status'].unique()

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'Не женат / не замужем'], dtype=object)

In [22]:
data['family_status'] = data['family_status'].str.lower()
data['family_status'].unique()

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'не женат / не замужем'], dtype=object)

#### Прочие

Посмотрим на данные, убедимся что статистики соответствуют нашим представлениям.

In [23]:
# OK
data[['dob_years','education_id', 'family_status_id', 'gender', 'debt']].describe() 

,dob_years,education_id,family_status_id,debt
count,21525.000000,21525.000000,21525.000000,21525.000000
mean,43.293380,0.817236,0.972544,0.080883
std,12.574584,0.548138,1.420324,0.272661
min,0.000000,0.000000,0.000000,0.000000
25%,33.000000,1.000000,0.000000,0.000000
50%,42.000000,1.000000,0.000000,0.000000
75%,53.000000,1.000000,1.000000,0.000000
max,75.000000,4.000000,4.000000,1.000000


In [24]:
data['gender'].value_counts() # OK

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

In [25]:
data['income_type'].unique() #OK

array(['сотрудник', 'пенсионер', 'компаньон', 'госслужащий',
       'безработный', 'предприниматель', 'студент', 'в декрете'],
      dtype=object)

In [26]:
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу


<a id="remove_dublicates_id"></a>
### Удаление дубликатов  

Данные изучены, нормализованы, удалим дубликаты

In [27]:
data.duplicated().sum()

71

In [28]:
data = data.drop_duplicates().reset_index(drop=True)
data.duplicated().sum()

0

In [29]:
data.duplicated().sum()

0

<a id="target_cat_id"></a>
### Категоризация целей кредита

В целях кредита наблюдается большое количество вариантов одного и того же значения. Поправим это. 

Согласно требованию учебного проекта, я не буду править это in-place, а заведу новые столбцы)

In [30]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

In [31]:
def credit_target_categorization(target):
    if 'авто' in target:
        return 'операции с автомобилем'
    
    if 'жил' in target or 'недвиж' in target:
        return 'операции с недвижимостью'
    
    if 'обра' in target:
        return 'получение образования'
    
    if 'свад' in target:
        return 'проведение свадьбы'
    
    
data['purpose_category'] = data['purpose'].apply(credit_target_categorization)

In [32]:
data['purpose_category'].unique()

array(['операции с недвижимостью', 'операции с автомобилем',
       'получение образования', 'проведение свадьбы'], dtype=object)

<a id="income_cat_id"></a>
### Категоризация дохода

Дескретизуем доход и категоризуем его буквенными обозначениями:

In [33]:
def income_categorization(income):
    if 0 <= income <= 30000:
        return 'E'
    
    if 30001 <= income <= 50000:
        return 'D'
    
    if 50001 <= income <= 200000:
        return 'C'
    
    if 200001 <= income <=  1000000:
        return 'B'
    
    if 1000001 <= income:
        return 'A'
    
    return 'Uncknown'

data['total_income_category'] = data['total_income'].apply(income_categorization)
data['total_income_category'].unique()

array(['B', 'C', 'E', 'D', 'A'], dtype=object)

<a id="additional_dataframe_id"></a>
### Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма.

Создаем два новых датафрейма, в которых:

- каждому уникальному значению из `education` соответствует уникальное значение `education_id` в первом;

- каждому уникальному значению из `family_status` соответствует уникальное значение `family_status_id` — во втором.


Потом удаляем из исходного датафрейма столбцы `education` и f`amily_status`, оставив только их идентификаторы: `education_id` и `family_status_id`. 

In [34]:
education_data = data[['education_id','education']].drop_duplicates().reset_index(drop=True)
education_data

,education_id,education
0,0,высшее
1,1,среднее
2,2,неоконченное высшее
3,3,начальное
4,4,ученая степень


In [35]:
family_data = data[['family_status_id','family_status']].drop_duplicates().reset_index(drop=True)
family_data

,family_status_id,family_status
0,0,женат / замужем
1,1,гражданский брак
2,2,вдовец / вдова
3,3,в разводе
4,4,не женат / не замужем


In [36]:
data.drop(['family_status', 'education'], axis=1, inplace=True)

In [37]:
data.head()

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,purpose_category,total_income_category
0,1,8437.673028,42,0,0,F,сотрудник,0,253875,покупка жилья,операции с недвижимостью,B
1,1,4024.803754,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,операции с автомобилем,C
2,0,5623.422610,33,1,0,M,сотрудник,0,145885,покупка жилья,операции с недвижимостью,C
3,3,4124.747207,32,1,0,M,сотрудник,0,267628,дополнительное образование,получение образования,B
4,0,340266.072047,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,проведение свадьбы,C


<a id="answer_id"></a>
## Ответы на вопросы

- Есть ли зависимость между количеством детей и возвратом кредита в срок?
- Есть ли зависимость между семейным положением и возвратом кредита в срок?
- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
- Как разные цели кредита влияют на его возврат в срок?

### Есть ли зависимость между количеством детей и возвратом кредита в срок?

In [38]:
children_data1 = data.pivot_table(index='children', columns='debt', values='purpose_category', aggfunc='count')

children_data1['ratio'] = children_data1[1] / children_data1[0] * 100
children_data1

debt,0,1,ratio
children,,,
0,13028.0,1063.0,8.159349
1,4410.0,445.0,10.090703
2,1926.0,202.0,10.488058
3,303.0,27.0,8.910891
4,37.0,4.0,10.810811
5,9.0,NaN,NaN


In [39]:
children_data = data.pivot_table(index='children',values='debt',aggfunc=['sum', 'count','mean'])
children_data['mean'] = children_data['mean']*100
children_data

,sum,count,mean
,debt,debt,debt
children,,,
0,1063,14091,7.543822
1,445,4855,9.165808
2,202,2128,9.492481
3,27,330,8.181818
4,4,41,9.756098
5,0,9,0.000000


~~В среднем процент людей, имеющих долги по кредиту в каждой категории, колеблется около 9-10%. 
У людей без детей, вероятность ниже на ~0.9 процентных пункта~~

~~**Нет существенной зависимости между количеством детей и возвратом кредита в срок**~~

Доля должников без детей существенно ниже. Возмножно это связанно с трудностью финансового планирования в услових неопределенности объема потребностей детей)

**Есть зависимость между количеством детей и возвратом кредита в срок**

### Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [40]:
family_status_data = data.pivot_table(index='family_status_id',values='debt',aggfunc=['sum', 'count','mean'])
family_status_data['mean'] = family_status_data['mean']*100

family_status_data

,sum,count,mean
,debt,debt,debt
family_status_id,,,
0,931,12339,7.545182
1,388,4151,9.347145
2,63,959,6.569343
3,85,1195,7.112971
4,274,2810,9.750890


Вдовы/вдовцы и люди в разводе реже имеют задолжностьпо кредиту - возможно они расчитывают только на себя и лучше оценивают свое финансовое положение

Следующимим идут люди в браке

В зоне риска находятся неженатые и живущие гражданским браком.

Разница между категориями до 2 проценных пунктов, 
**можно сказать,что есть зависимость между семейным положением и возвратом кредита в срок**


In [41]:
family_data

,family_status_id,family_status
0,0,женат / замужем
1,1,гражданский брак
2,2,вдовец / вдова
3,3,в разводе
4,4,не женат / не замужем


### Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [42]:
total_income_data = data.pivot_table(index='total_income_category', columns='debt', values='purpose_category', aggfunc='count')

total_income_data['ratio'] = total_income_data[1] / total_income_data[0] * 100
total_income_data

debt,0,1,ratio
total_income_category,,,
A,23,2,8.695652
B,4685,356,7.598719
C,12723,1190,9.353140
D,329,21,6.382979
E,1953,172,8.806964


Можно увидеть что лучше всех возвращают кредиты люди со средним уровнем дохода (категория D)
и люди с высоким уровнем дохода (категория B)


Люди с супервысоким, супернизким и средними доходами (A С и D)возвращают кредиты приблизительно одинаково.

Можно утвержать что **есть зависимость между уровнем дохода и возвратом кредита в срок**
но характер заисимости интересный)

### Как разные цели кредита влияют на его возврат в срок?

In [43]:
purpose_category_data = data.pivot_table(index='purpose_category', columns='debt', values='purpose', aggfunc='count')

purpose_category_data['ratio'] = purpose_category_data[1] / purpose_category_data[0] * 100
purpose_category_data

debt,0,1,ratio
purpose_category,,,
операции с автомобилем,3903,403,10.325391
операции с недвижимостью,10029,782,7.797388
получение образования,3643,370,10.156464
проведение свадьбы,2138,186,8.699719


Лучше всего отдают кредиты за свадьбу и недвижимость, в случае покупки автомобиля и получение образования проблем возникает больше. Можно предположить, что это связанно с размером кредита в случае свадьбы и его продуманностью, в случае покупки недвижимости

Можно уттверждать, что есть зависимость между возвратом кредита и целями, на который он взят.


<a id="common_answer_id"></a>
## Общий вывод:

При выдаче кредита банку стоит обращать внимание на семейное положение и цели, на которые берется кредит. Также следует учитывать уровень дохода клиента. Отсутствие детей в семье детей повышает надежность клиента.